In [2]:
import pandas as pd
import os
import requests
import boto3
from utils import *
import datetime

**EXTRACTION**

GET SUMMONER PUUID BY SUMMONER NAME AND TAGLINE

In [3]:
api_key = os.environ.get("ETL-LOL_API")

summ_name = "TATIAN"
tag_line = "LAS"

params = {
    'api_key': api_key
}

servers = {
    'AMERICAS': 'americas.api.riotgames.com',
    'ASIA': 'asia.api.riotgames.com',
    'EUROPE': 'europe.api.riotgames.com',   
    'SEA': 'sea.api.riotgames.com'
}

endpoint = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{summ_name}/{tag_line}"

res = requests.get(endpoint, params= params)
data = res.json()

puuid = data['puuid']

df_summoner = pd.json_normalize(data)

df_summoner

,puuid,gameName,tagLine
0,rybLSc_KZF6-IQ8HCLMML90_kvd00YLW5DE0ruoXGgZF2P...,TATIAN,LAS


GET SUMMONER DATA BY PUUID

In [4]:
endpoint = f"https://la2.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"

res = requests.get(endpoint, params=params)
data = dict(res.json())

summ_id = data['id']

data.pop('puuid')

df_summoner = pd.concat([df_summoner, pd.json_normalize(data)], axis=1)

df_summoner = df_summoner.drop(columns=['profileIconId'])

df_summoner

,puuid,gameName,tagLine,id,accountId,revisionDate,summonerLevel
0,rybLSc_KZF6-IQ8HCLMML90_kvd00YLW5DE0ruoXGgZF2P...,TATIAN,LAS,lJDYoZSk-rtpKy1UnGV65cDlknAvnLOq8uvnGRFFCjIwHw,3SsoFKJ2HArAyEROZF7-cn0zHxd1m-rxRYxt9MScpr1oaME,1731213102000,362


GET CHAMPIONS LIST

In [5]:
champions_endpoint = "https://ddragon.leagueoflegends.com/cdn/14.22.1/data/en_US/champion.json"

data = requests.get(champions_endpoint).json()

df_champions = pd.DataFrame(data['data'])

df_champions = df_champions.T

df_champions.head(1)

,version,id,key,name,title,blurb,info,image,tags,partype,stats
Aatrox,14.22.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'full': 'Aatrox.png', 'sprite': 'champion0.pn...",[Fighter],Blood Well,"{'hp': 650, 'hpperlevel': 114, 'mp': 0, 'mpper..."


GET MASTERY CHAMPIONS BY SUMMONER PUUID

In [6]:
endpoint = f"https://la2.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}"

res = requests.get(endpoint, params=params)
data = res.json()

df_champions_mastery = pd.json_normalize(data)

df_champions_mastery.head(1)


,puuid,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,markRequiredForNextLevel,tokensEarned,championSeasonMilestone,nextSeasonMilestone.requireGradeCounts.B-,nextSeasonMilestone.requireGradeCounts.C-,nextSeasonMilestone.rewardMarks,nextSeasonMilestone.bonus,nextSeasonMilestone.rewardConfig.rewardValue,nextSeasonMilestone.rewardConfig.rewardType,nextSeasonMilestone.rewardConfig.maximumReward,nextSeasonMilestone.totalGamesRequires,milestoneGrades,nextSeasonMilestone.requireGradeCounts.A-
0,rybLSc_KZF6-IQ8HCLMML90_kvd00YLW5DE0ruoXGgZF2P...,64,32,372982,1717130755000,55382,-44382,2,0,0,1.0,4,1,False,5f4333db-e90d-4705-903b-08dbf5e61006,HEXTECH_CHEST,6.0,5,NaN,NaN


GET THE LAST 20 MATCHES:
1- FIRST I GET THE MATCHES IDs.
2- SINCE FROM THOSE IDs, I CAN GET MATCHES DATA 

In [305]:
endpoint = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"

res = requests.get(endpoint, params=params)
matches_id = res.json()

matches_id

matches = []
for matchId in matches_id:
    endpoint = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}"
    
    res = requests.get(endpoint, params=params)
    matches.append(res.json())
    
df_matches = pd.json_normalize(matches)

df_matches.head(1)

,metadata.dataVersion,metadata.matchId,metadata.participants,info.endOfGameResult,info.gameCreation,info.gameDuration,info.gameEndTimestamp,info.gameId,info.gameMode,info.gameName,info.gameStartTimestamp,info.gameType,info.gameVersion,info.mapId,info.participants,info.platformId,info.queueId,info.teams,info.tournamentCode
0,2,LA2_1460246484,[lmY9N2GCcOkNkiEn2FCEcUSbMx0njx137z_JYJ70BjGq5...,GameComplete,1731212021831,971,1731213098902,1460246484,CLASSIC,teambuilder-match-1460246484,1731212127539,MATCHED_GAME,14.22.633.1362,11,"[{'allInPings': 0, 'assistMePings': 1, 'assist...",LA2,420,"[{'bans': [{'championId': 910, 'pickTurn': 1},...",


MATCHES DETAILS

In [306]:
df_matches_details = df_matches[["metadata.matchId", "metadata.participants","info.participants","info.teams"]]

df_matches = df_matches.drop(columns=["metadata.participants","info.participants","info.teams"])

In [316]:
df_matches_details = df_matches_details.explode(["metadata.participants", "info.participants"]).reset_index(drop=True)

df_info_participants = pd.json_normalize(df_matches_details["info.participants"])
df_info_teams = pd.json_normalize(df_matches_details["info.teams"])

df_matches_details = df_matches_details.drop(columns=["info.participants", "info.teams"]).reset_index(drop=True)

df_matches_details = pd.concat([df_matches_details,
                                df_info_participants,
                                df_info_teams], axis= 1)

#REMOVE DUPLICATED COLUMNS
df_matches_details = df_matches_details.loc[:,~df_matches_details.columns.duplicated()].copy()

#REMOVE DUPLICATED ROWS
df_matches_details = df_matches_details.loc[df_matches_details.astype(str).drop_duplicates().index]

In [327]:
df_matches_details.head(10)
#df_matches_details.to_parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/matches_details.parquet", engine='pyarrow', index=False)

,metadata.matchId,metadata.participants,allInPings,assistMePings,assists,baronKills,basicPings,bountyLevel,champExperience,champLevel,...,challenges.earliestBaron,challenges.fastestLegendary,challenges.teleportTakedowns,challenges.shortestTimeToAceFromFirstTakedown,challenges.thirdInhibitorDestroyedTime,challenges.baronBuffGoldAdvantageOverThreshold,challenges.hadAfkTeammate,challenges.earliestElderDragon,0,1
0,LA2_1460246484,lmY9N2GCcOkNkiEn2FCEcUSbMx0njx137z_JYJ70BjGq5-...,0,1,2,0,0,4,7372,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'bans': [{'championId': 910, 'pickTurn': 1}, ...","{'bans': [{'championId': 25, 'pickTurn': 6}, {..."
1,LA2_1460246484,2eviXPb15yLyYImQPHZg5BAho1RhurVGJasRo5VZ-FSoe7...,0,4,4,0,0,3,5098,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'bans': [{'championId': 910, 'pickTurn': 1}, ...","{'bans': [{'championId': 25, 'pickTurn': 6}, {..."
2,LA2_1460246484,vGmT00-3fgI1ewJb0TJZRWKFYJco-KFYcLUO1zaw_hjuxg...,0,0,2,0,0,2,6343,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'bans': [{'championId': 910, 'pickTurn': 1}, ...","{'bans': [{'championId': 25, 'pickTurn': 6}, {..."
3,LA2_1460246484,hBbuMcrUxSs_iKTEy-mjEA6dFIGRY41QKORqgn4PxRbM8u...,0,0,0,0,0,1,5247,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'bans': [{'championId': 910, 'pickTurn': 1}, ...","{'bans': [{'championId': 25, 'pickTurn': 6}, {..."
4,LA2_1460246484,c_G2iJohPgy7WeovnO6RJcDI_lOhvT1AxkUndS4F_lslBA...,0,12,6,0,0,0,4421,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'bans': [{'championId': 910, 'pickTurn': 1}, ...","{'bans': [{'championId': 25, 'pickTurn': 6}, {..."
5,LA2_1460246484,-FOe5DB-g_H_oDBUdE0N-5diMHWzq7mVyW4OEXaGm8MooI...,0,0,3,0,0,0,7681,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'bans': [{'championId': 910, 'pickTurn': 1}, ...","{'bans': [{'championId': 25, 'pickTurn': 6}, {..."
6,LA2_1460246484,oEzWxmH7i1c_K3n6rABE9BFuU_ZKML7kgEZJvXx6qoB9aW...,0,0,4,0,0,11,7845,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'bans': [{'championId': 910, 'pickTurn': 1}, ...","{'bans': [{'championId': 25, 'pickTurn': 6}, {..."
7,LA2_1460246484,osRupWA12cC1otfwC2jmZtTuV4-_lCSjVcTl7X_w190Mnk...,2,2,6,0,0,6,9893,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'bans': [{'championId': 910, 'pickTurn': 1}, ...","{'bans': [{'championId': 25, 'pickTurn': 6}, {..."
8,LA2_1460246484,rybLSc_KZF6-IQ8HCLMML90_kvd00YLW5DE0ruoXGgZF2P...,0,3,3,0,0,0,6563,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'bans': [{'championId': 910, 'pickTurn': 1}, ...","{'bans': [{'championId': 25, 'pickTurn': 6}, {..."
9,LA2_1460246484,FyjJwSZX7yz2DwLtzG7hMNLMKzuyNmTd5si4Jp4jXopNLt...,0,1,2,0,0,2,3165,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'bans': [{'championId': 910, 'pickTurn': 1}, ...","{'bans': [{'championId': 25, 'pickTurn': 6}, {..."


GET SUMMONER RANKING BY SUMMONER ID

In [10]:
endpoint = f"https://la2.api.riotgames.com/lol/league/v4/entries/by-summoner/{summ_id}"

res = requests.get(endpoint, params=params)

data = res.json()

df_ranking = pd.json_normalize(data)

df_ranking

,leagueId,queueType,tier,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,6d220f29-c66d-4999-8bee-fc750c55fb2e,RANKED_SOLO_5x5,PLATINUM,III,lJDYoZSk-rtpKy1UnGV65cDlknAvnLOq8uvnGRFFCjIwHw,60,6,3,False,False,False,False
1,67df7375-00af-4a4f-9282-761ec6d3ca50,RANKED_FLEX_SR,EMERALD,IV,lJDYoZSk-rtpKy1UnGV65cDlknAvnLOq8uvnGRFFCjIwHw,71,4,7,False,False,False,False


GET ITEMS LIST

**LOAD RAW DATA TO DATA LAKE (S3)**

CREATING S3 CLIENT

In [10]:
s3_client = boto3.client('s3', region_name='us-east-1')

CREATING BUCKET

In [38]:
#CREATING A BUCKET
s3_client.create_bucket(ACL='private', Bucket='mylolapibucket1')

{'ResponseMetadata': {'RequestId': 'VAVAP8FTVFYBJAAP',
  'HostId': 'cwmG4sQJGiHcjoJodQo5ypggKUXFRZGw68LH3lXrKhFaxlffhkqbgp2hijpv2ZCAiMOqqjgCGCE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'cwmG4sQJGiHcjoJodQo5ypggKUXFRZGw68LH3lXrKhFaxlffhkqbgp2hijpv2ZCAiMOqqjgCGCE=',
   'x-amz-request-id': 'VAVAP8FTVFYBJAAP',
   'date': 'Mon, 25 Nov 2024 01:31:23 GMT',
   'location': '/mylolapibucket1',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/mylolapibucket1'}

LOADING DATA

In [39]:
#PATH
s3_path = f"lol_data/{datetime.date.today().strftime("%Y-%m-%d")}"

upload_dataframe_to_s3(s3_client, df_summoner, f"{s3_path}/summoner.parquet", "mylolapibucket1")
upload_dataframe_to_s3(s3_client, df_ranking, f"{s3_path}/ranking.parquet", "mylolapibucket1")
upload_dataframe_to_s3(s3_client, df_champions, f"{s3_path}/champions.parquet", "mylolapibucket1")
upload_dataframe_to_s3(s3_client, df_champions_mastery, f"{s3_path}/champions_mastery.parquet", "mylolapibucket1")
upload_dataframe_to_s3(s3_client, df_matches, f"{s3_path}/matches.parquet", "mylolapibucket1")
upload_dataframe_to_s3(s3_client, df_matches_details, f"{s3_path}/matches_details.parquet", "mylolapibucket1")
upload_dataframe_to_s3(s3_client, df_matches_timeline, f"{s3_path}/matches_timeline.parquet", "mylolapibucket1")

**TRANSFORMATION**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, from_unixtime, explode, from_json, concat_ws, round
import findspark

CREATING SPARK SESSION

In [ ]:
findspark.init()
spark_session = SparkSession.builder.appName("LOL_API_MODERN_DATAWAREHOUSE").getOrCreate()

GETTING PARQUETS FROM DATA LAKE (S3)

In [ ]:
df_summoner_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/summoner.parquet"))
df_ranking_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/ranking.parquet"))
df_champions_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/champions.parquet"))
df_champions_mastery_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/champions_mastery.parquet"))
df_matches_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/matches.parquet"))
df_matches_details_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/matches_details.parquet"))
df_matches_timeline_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/matches_timeline.parquet"))

SUMMONER TRANSFORM

In [ ]:
#JOIN SUMMONER DF AND RANKING DF
df_summoner_spark = df_summoner_spark.join(df_ranking_spark, on=df_summoner_spark["id"] == df_ranking_spark["summonerId"])
df_summoner_spark = df_summoner_spark.drop("id")

#CONVERT UNIX TIMESTAMP TO DATE
df_summoner_spark = df_summoner_spark.withColumn("revisionDate", from_unixtime(col("revisionDate") / 1000).cast("timestamp"))

columns = df_summoner_spark.columns

summoner_puuid = columns.index("puuid")
summoner_summonerId = columns.index("summonerId")

columns[summoner_puuid], columns[summoner_summonerId] = columns[summoner_summonerId], columns[summoner_puuid]

df_summoner_spark = df_summoner_spark.select(columns)

In [15]:
df_summoner_spark.show()

+--------------------+--------+-------+--------------------+--------------------+-------------------+-------------+--------------------+---------------+--------+----+--------------------+------------+----+------+-------+--------+----------+---------+
|          summonerId|gameName|tagLine|                  id|           accountId|       revisionDate|summonerLevel|            leagueId|      queueType|    tier|rank|               puuid|leaguePoints|wins|losses|veteran|inactive|freshBlood|hotStreak|
+--------------------+--------+-------+--------------------+--------------------+-------------------+-------------+--------------------+---------------+--------+----+--------------------+------------+----+------+-------+--------+----------+---------+
|lJDYoZSk-rtpKy1Un...|  TATIAN|    LAS|lJDYoZSk-rtpKy1Un...|3SsoFKJ2HArAyEROZ...|2024-11-10 01:31:42|          362|6d220f29-c66d-499...|RANKED_SOLO_5x5|PLATINUM| III|rybLSc_KZF6-IQ8HC...|          60|   6|     3|  false|   false|     false|    fal

CHAMPIONS TRANSFORM

In [ ]:
#LIST TO STRING
df_champions_spark = df_champions_spark.withColumn("tags", concat_ws(", ", col("tags")))

#EXPLODE INFO FIELD
df_champions_spark = df_champions_spark.select("*",
                                                 col("info.attack").alias("attack"),
                                                 col("info.defense").alias("defense"),
                                                 col("info.magic").alias("magic"),
                                                 col("info.difficulty").alias("difficulty"),
                                                 col("stats.hp").alias("hp"),
                                                 col("stats.hpperlevel").alias("hpPerLevel"),
                                                 col("stats.mp").alias("mp"),
                                                 col("stats.mpperlevel").alias("mpPerLevel"),
                                                 col("stats.movespeed").alias("moveSpeed"),
                                                 col("stats.armor").alias("armor"),
                                                 col("stats.armorperlevel").alias("armorPerLevel"),
                                                 col("stats.spellblock").alias("spellBlock"),
                                                 col("stats.spellblockperlevel").alias("spellBlockPerLevel"),
                                                 col("stats.attackrange").alias("attackRange"),
                                                 col("stats.hpregen").alias("hpRegen"),
                                                 col("stats.hpregenperlevel").alias("hpRegenPerLevel"),
                                                 col("stats.mpregen").alias("mpRegen"),
                                                 col("stats.mpregenperlevel").alias("mpRegenPerLevel"),
                                                 col("stats.crit").alias("crit"),
                                                 col("stats.critperlevel").alias("critPerLevel"),
                                                 col("stats.attackdamage").alias("attackDamage"),
                                                 col("stats.attackdamageperlevel").alias("attackDamagePerLevel"),
                                                 col("stats.attackspeedperlevel").alias("attackSpeedPerLevel"),
                                                 col("stats.attackspeed").alias("attackSpeed"))

df_champions_spark = df_champions_spark.drop("info", "image", "stats", "id")

df_champions_spark = df_champions_spark.withColumnRenamed("key", "championId")

In [17]:
df_champions_spark.show(2)

+-------+----------+------+-------------------+--------------------+--------------+----------+------+-------+-----+----------+---+----------+---+----------+---------+-----+-------------+----------+------------------+-----------+-------+---------------+-------+---------------+----+------------+------------+--------------------+-------------------+-----------+
|version|championId|  name|              title|               blurb|          tags|   partype|attack|defense|magic|difficulty| hp|hpPerLevel| mp|mpPerLevel|moveSpeed|armor|armorPerLevel|spellBlock|spellBlockPerLevel|attackRange|hpRegen|hpRegenPerLevel|mpRegen|mpRegenPerLevel|crit|critPerLevel|attackDamage|attackDamagePerLevel|attackSpeedPerLevel|attackSpeed|
+-------+----------+------+-------------------+--------------------+--------------+----------+------+-------+-----+----------+---+----------+---+----------+---------+-----+-------------+----------+------------------+-----------+-------+---------------+-------+---------------+--

CHAMPIONS MASTERY TRANSFORM

In [18]:
df_champions_mastery_spark = df_champions_mastery_spark.withColumn("lastPlayTime", from_unixtime(col("lastPlayTime") / 1000).cast("timestamp"))

df_champions_mastery_spark = df_champions_mastery_spark.drop("championPointsSinceLastLevel",
                                                             "championPointsUntilNextLevel",
                                                             "markRequiredForNextLevel",
                                                             "tokensEarned",
                                                             "championSeasonMilestone",
                                                             "nextSeasonMilestone.requireGradeCounts.B-",
                                                             "nextSeasonMilestone.requireGradeCounts.C-",
                                                             "nextSeasonMilestone.rewardMarks",
                                                             "nextSeasonMilestone.bonus",
                                                             "nextSeasonMilestone.rewardConfig.rewardValue",
                                                             "nextSeasonMilestone.rewardConfig.rewardType",
                                                             "nextSeasonMilestone.rewardConfig.maximumReward",
                                                             "nextSeasonMilestone.totalGamesRequires",
                                                             "milestoneGrades",
                                                             "nextSeasonMilestone.requireGradeCounts.A-")

In [19]:
df_champions_mastery_spark.show(2)

+--------------------+----------+-------------+--------------+-------------------+
|               puuid|championId|championLevel|championPoints|       lastPlayTime|
+--------------------+----------+-------------+--------------+-------------------+
|rybLSc_KZF6-IQ8HC...|        64|           32|        372982|2024-05-31 01:45:55|
|rybLSc_KZF6-IQ8HC...|       412|           15|        177802|2024-04-24 02:33:12|
+--------------------+----------+-------------+--------------+-------------------+
only showing top 2 rows



MATCHES TRANSFORM

In [20]:
df_matches_spark = df_matches_spark.withColumnsRenamed({"metadata.dataVersion":"dataVersion",
                                                        "metadata.matchId":"matchId",
                                                        "info.endOfGameResult":"endOfGameResult",
                                                        "info.gameCreation":"gameCreation",
                                                        "info.gameEndTimestamp":"gameEndTimestamp",
                                                        "info.gameDuration":"gameDuration(mins)",
                                                        "info.gameId":"gameId",
                                                        "info.gameMode":"gameMode",
                                                        "info.gameName":"gameName",
                                                        "info.gameStartTimestamp":"gameStartTimestamp",
                                                        "info.gameType":"gameType",
                                                        "info.gameVersion":"gameVersion",
                                                        "info.mapId":"mapId",
                                                        "info.platformId":"platformId",
                                                        "info.queueId":"queueId",
                                                        "info.tournamentCode":"tournamentCode",
                                                        })

df_matches_spark = df_matches_spark.withColumn("gameCreation", from_unixtime(col("gameCreation") / 1000).cast("timestamp"))
df_matches_spark = df_matches_spark.withColumn("gameEndTimestamp", from_unixtime(col("gameEndTimestamp") / 1000).cast("timestamp"))
df_matches_spark = df_matches_spark.withColumn("gameStartTimestamp", from_unixtime(col("gameStartTimestamp") / 1000).cast("timestamp"))
df_matches_spark = df_matches_spark.withColumn("gameDuration(mins)", round((col("gameDuration(mins)") / 60).cast("double"), 2))

In [21]:
df_matches_spark.show(1)

+-----------+--------------+---------------+-------------------+------------------+-------------------+----------+--------+--------------------+-------------------+------------+--------------+-----+----------+-------+--------------+
|dataVersion|       matchId|endOfGameResult|       gameCreation|gameDuration(mins)|   gameEndTimestamp|    gameId|gameMode|            gameName| gameStartTimestamp|    gameType|   gameVersion|mapId|platformId|queueId|tournamentCode|
+-----------+--------------+---------------+-------------------+------------------+-------------------+----------+--------+--------------------+-------------------+------------+--------------+-----+----------+-------+--------------+
|          2|LA2_1460246484|   GameComplete|2024-11-10 01:13:41|             16.18|2024-11-10 01:31:38|1460246484| CLASSIC|teambuilder-match...|2024-11-10 01:15:27|MATCHED_GAME|14.22.633.1362|   11|       LA2|    420|              |
+-----------+--------------+---------------+-------------------+----

MATCHES DETAILS

In [338]:
df_matches_details_spark = df_matches_details_spark.withColumnsRenamed({"metadata.matchId":"matchId",
                                                                        "metadata.participants":"participant_id"})

cols_to_select = [
    'puuid',
    'summonerId',
    'participant_id',
    'matchId',
    'teamId',
    'summonerName',
    'summonerLevel',
    'win',
    'lane',
    'role',
    'teamPosition',
    'individualPosition',
    'championId',
    'championName',
    'champLevel',
    'kills',
    'deaths',
    'assists',
    'totalMinionsKilled',
    'goldEarned',
    'goldSpent',
    'timePlayed',
    'doubleKills',
    'tripleKills',
    'quadraKills',
    'pentaKills',
    'killingSprees',
    'wardsKilled',
    'wardsPlaced',
    'visionWardsBoughtInGame',
    'visionScore',
    'dragonKills',
    'turretKills',
    'inhibitorKills',
    'nexusKills',
    'inhibitorTakedowns',
    'inhibitorsLost',
    'detectorWardsPlaced',
    'objectivesStolen',
    'itemsPurchased',
    'item0',
    'item1',
    'item2',
    'item3',
    'item4',
    'item5',
    'item6'
]

df_matches_details_spark = df_matches_details_spark.select(cols_to_select)

df_matches_details_spark = df_matches_details_spark.withColumn("timePlayed", round(col("timePlayed") / 60, 2))

df_matches_details_spark = df_matches_details_spark.withColumnRenamed("timePlayed", "timePlayed(mins)")

In [346]:
df_matches_details_spark.show(10,truncate=False)

+------------------------------------------------------------------------------+----------------------------------------------------------+------------------------------------------------------------------------------+--------------+------+------------+-------------+-----+----+-------+------------+------------------+----------+------------+----------+-----+------+-------+------------------+----------+---------+----------------+-----------+-----------+-----------+----------+-------------+-----------+-----------+-----------------------+-----------+-----------+-----------+--------------+----------+------------------+--------------+-------------------+----------------+--------------+-----+-----+-----+-----+-----+-----+-----+
|puuid                                                                         |summonerId                                                |participant_id                                                                |matchId       |teamId|summonerName|summonerLevel|wi

ITEMS

In [347]:
df_matches_timeline_spark.show(1)

+--------------------+----------------+---------------------+--------------------+------------------+--------------------+-----------+--------------------+
|metadata.dataVersion|metadata.matchId|metadata.participants|info.endOfGameResult|info.frameInterval|         info.frames|info.gameId|   info.participants|
+--------------------+----------------+---------------------+--------------------+------------------+--------------------+-----------+--------------------+
|                   2|  LA2_1460246484| [lmY9N2GCcOkNkiEn...|        GameComplete|             60000|[{[{NULL, NULL, N...| 1460246484|[{1, lmY9N2GCcOkN...|
+--------------------+----------------+---------------------+--------------------+------------------+--------------------+-----------+--------------------+
only showing top 1 row



In [13]:
df_summoner_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/summoner.parquet")
df_ranking_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/ranking.parquet")
df_champions_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/champions.parquet")
df_champions_mastery_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/champions_mastery.parquet")
df_matches_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/matches.parquet")
df_matches_details_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/matches_details.parquet")
df_matches_timeline_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/matches_timeline.parquet")

In [337]:
df_matches_details_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/matches_details.parquet")